In [1]:
import ee
import geemap as emap

# emap.update_package()

**Initialize and authenticate**

In [2]:
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/1AX4XfWgXWmUB9q4eYT72ISqFsvTgp8D03cg5N_fSMfjWljbIxdkWa00zEyg

Successfully saved authorization token.


In [3]:
Map=emap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

**Load the Sentinel-2 Surface Reflectance**

In [28]:
# Load the study area
area=ee.FeatureCollection("users/miketu72/Test_Area")
point=ee.FeatureCollection("users/miketu72/Test_point")
Map.centerObject(area,10)
Map.addLayer(point,{},"Area")

In [8]:
# Load sentinel-2 data from 2020. Please keep in mind the factor =0.0001
sentinel2=ee.ImageCollection("COPERNICUS/S2_SR").filterDate("2020-01-01","2020-12-31").\
filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than",30).filterBounds(area).first().clip(area)

In [9]:
Map.centerObject(area,10)
Map.addLayer(sentinel2,{"bands":["B8","B4","B3"],"min":780.2, "max":3189.2},"Sentinel")  # 651-1448, 762-2077
Map

Map(bottom=230698.0, center=[21.242275458817613, 105.49860474289395], controls=(WidgetControl(options=['positi…

**Calculate NDVI**

In [25]:
NDVI=sentinel2.expression("(NIR-RED)/(NIR+RED)",\
                          {"NIR":sentinel2.select("B8"),\
                          "RED":sentinel2.select("B4")}).rename("NDVI")
# Add a NDVI band to the Sentinel-2 image
sentinel2=sentinel2.addBands(NDVI)

**Extract raster values using point shapefile**

In [11]:
scale = 10  # scale in meters

# Print the Sentinel 
elv_urban_point = sentinel2.sample(point, scale).first().get('B4').getInfo()
print('value:', elv_urban_point)

value: 1854


**Time-series Data Extraction**

In [12]:
sen2=ee.ImageCollection("COPERNICUS/S2_SR").filterDate("2020-01-01","2020-12-31").\
filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than",30).filterBounds(area)

In [13]:
data = sen2.getRegion(point,scale).getInfo()

In [18]:
import pandas as pd
df=pd.DataFrame(data[1:], columns=data[0])
df

,id,longitude,latitude,time,B1,B2,B3,B4,B5,B6,...,WVP,SCL,TCI_R,TCI_G,TCI_B,MSK_CLDPRB,MSK_SNWPRB,QA10,QA20,QA60
0,20200218T032811_20200218T033333_T48QWJ,105.524692,21.164353,1581997330480,568,512,830,630,1030,2693,...,1617,4,67,85,57,2,0,0,0,0
1,20200309T032551_20200309T033758_T48QWJ,105.524692,21.164353,1583725332600,736,734,974,709,1100,3028,...,2252,4,72,100,75,1,0,0,0,0
2,20200428T032541_20200428T033028_T48QWJ,105.524692,21.164353,1588045339021,527,473,775,528,1150,2983,...,2967,4,54,79,51,0,0,0,0,0
3,20200518T032541_20200518T033031_T48QWJ,105.524692,21.164353,1589773341883,535,557,843,710,1214,3403,...,4771,4,75,87,57,0,0,0,0,0
4,20200622T032539_20200622T033715_T48QWJ,105.524692,21.164353,1592797339103,287,365,662,438,936,3477,...,5556,4,42,67,35,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,20200518T032541_20200518T033031_T48QWJ,105.502594,21.316617,1589773341883,600,581,830,599,1176,982,...,4369,2,60,85,58,0,0,0,0,0
380,20200622T032539_20200622T033715_T48QWJ,105.502594,21.316617,1592797339103,513,446,641,441,867,523,...,5244,2,44,64,48,0,0,0,0,0
381,20200826T032541_20200826T032643_T48QWJ,105.502594,21.316617,1598413341884,453,469,894,572,1373,1167,...,4029,2,57,90,47,0,0,0,0,0
382,20201109T032949_20201109T033934_T48QWJ,105.502594,21.316617,1604893338106,562,664,927,648,1049,659,...,2318,2,68,97,68,0,0,0,0,0


**Challenge** 

Calculate `NDVI` for each sentinel-2 image and `monthly NDVI` for a given study area.

In [33]:
s2=ee.ImageCollection("COPERNICUS/S2_SR").filterDate("2020-01-01","2020-12-31").filterBounds(area)

s2.size().getInfo()

73

In [34]:
def NDVI(image):
    ndvi=image.expression("(B8-B4)/(B8+B4)",{"B8":image.select("B8"),"B4":image.select("B4")}).rename("NDVI")
    return image.addBands(ndvi)

In [47]:
sa=s2.map(NDVI)

**Image Composite**

In [10]:
# Mosaic composite
s2=ee.ImageCollection("COPERNICUS/S2_SR").filterDate("2020-01-01","2020-12-31").filterBounds(area)

mosaic=s2.mosaic()

Map.addLayer(mosaic,{"bands":["B4","B3","B2"],"min":780.2, "max":3189.2},"Mosaic")
Map

Map(bottom=29213.0, center=[20.709877019887912, 106.248779296875], controls=(WidgetControl(options=['position'…

In [13]:
# Median Composite
median=s2.median()

Map.addLayer(median,{"bands":["B4","B3","B2"],"min":500.2, "max":2854.2},"Median")
Map

Map(bottom=462767.0, center=[20.755150751051744, 105.23838043212892], controls=(WidgetControl(options=['positi…

In [6]:
# Display the Hanoi boundary 
UN=ee.FeatureCollection("FAO/GAUL/2015/level2")

Singapore=UN.filterMetadata("ADM0_NAME","equals","Singapore")


In [5]:
# Obtain Landsat 8 TOA Level 1
LS8=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate("2018-01-01","2018-12-31")\
.filterMetadata("CLOUD_COVER","less_than",10).filterBounds(Singapore) # Filter boundary of Singapore and cloud cover
Map.addLayer(LS8,{},"LS8")
Map

Map(bottom=65340.0, center=[1.3611355787659067, 103.8202830927031], controls=(WidgetControl(options=['position…

In [7]:
Map.centerObject(Singapore,9)
Map.addLayer(ee.Image().paint(Singapore,0,1.5),{"palette":["red"]},"Singapore") # Colour = 0 or set color="red", and width =1.5
Map

Map(bottom=65340.0, center=[1.3611355787659067, 103.8202830927031], controls=(WidgetControl(options=['position…

In [8]:
# Obtain median over the Image Collection
LS_Sing=LS8.median().float().clip(Singapore) # Convert to float32 as it has both float32 and float64. Can't download if mixed
Map.addLayer(LS_Sing,{},"LSSIng")
Map

Map(bottom=65340.0, center=[1.3611355787659067, 103.8202830927031], controls=(WidgetControl(options=['position…

In [9]:
LS_Sing.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']

In [10]:
# Download a single image with boundary of Singapore and 30m pixel size
task = ee.batch.Export.image.toDrive(image=LS_Sing,  # an ee.Image object.
                                     region=Singapore.geometry().bounds().getInfo()["coordinates"],  # an ee.Geometry object.
                                     description="LS_Sing",
                                     folder="Tam_Example",
                                     fileNamePrefix="LS_Sing",
                                     scale=30)
task.start()

**Write a function to get a previous month image from the current date**

In [5]:
import datetime as dt
def imageMonth(date=input("Please enter your date in yyyy, mm,dd format (e.g., 2020,4,20)")):
    now=dt.date.now()

Please enter your date in yyyy, mm,dd format (e.g., 2020,4,20)2020,2,34
